In [7]:
import pandas as pd

# --- 1. Load World Bank-style multi-year data ---
def clean_world_bank_multi_year(path, indicator_name, years):
    df = pd.read_csv(path, skiprows=4)
    df = df[["Country Name"] + years]
    df = df.melt(id_vars="Country Name", value_vars=years,
                 var_name="Year", value_name=indicator_name)
    df = df.rename(columns={"Country Name": "Country"})
    df[indicator_name] = pd.to_numeric(df[indicator_name], errors="coerce")
    df["Country"] = df["Country"].str.strip().str.lower()
    return df

# --- 2. Load simpler formats like tax revenue or debt ---
def clean_simple_csv(path, country_col, value_name, years):
    df = pd.read_csv(path)
    df.replace("no data", pd.NA, inplace=True)
    df = df.rename(columns={country_col: "Country"})
    df = df[["Country"] + years]
    df = df.melt(id_vars="Country", value_vars=years,
                 var_name="Year", value_name=value_name)
    df[value_name] = pd.to_numeric(df[value_name], errors="coerce")
    df["Country"] = df["Country"].str.strip().str.lower()
    return df

# --- 3. Define years ---
years_range = [str(y) for y in range(2013, 2024)]

# --- 4. Load all datasets ---
debt         = clean_simple_csv("./data/debt/debt.csv", "Central Government Debt (Percent of GDP)", "Debt", years_range)
tax_revenue  = clean_simple_csv("./data/tax_revenue/tax_revenue.csv", "Government revenue, percent of GDP (% of GDP)", "Tax_revenue", years_range)
gdp          = clean_world_bank_multi_year("./data/GDP_per/GDP_per_capita.csv", "GDP_per_capita", years_range)
gni          = clean_world_bank_multi_year("./data/GNI/GNI_per_capita.csv", "GNI_per_capita", years_range)
population   = clean_world_bank_multi_year("./data/population/population.csv", "Population", years_range)
unemploy     = clean_world_bank_multi_year("./data/unemploy/unemploy.csv", "Unemployment", years_range)
inflation    = clean_world_bank_multi_year("./data/inflation/inflation.csv", "Inflation", years_range)
interest     = clean_world_bank_multi_year("./data/interest/interest.csv", "Interest", years_range)

# --- 5. Merge all datasets on Country and Year ---
df = debt.merge(tax_revenue, on=["Country", "Year"]) \
         .merge(gdp, on=["Country", "Year"]) \
         .merge(gni, on=["Country", "Year"]) \
         .merge(population, on=["Country", "Year"]) \
         .merge(unemploy, on=["Country", "Year"]) \
         .merge(inflation, on=["Country", "Year"]) \
         .merge(interest, on=["Country", "Year"])

# --- 6. Drop rows with missing values ---
df = df.dropna()

# --- 7. Categorize Income Group by GDP ---
def categorize_income(gdp):
    if gdp < 1085:
        return "Low income"
    elif gdp <= 4255:
        return "Lower-middle income"
    elif gdp <= 13205:
        return "Upper-middle income"
    else:
        return "High income"

df["Income_Category"] = df["GDP_per_capita"].apply(categorize_income)

# Optional: Save to CSV for Tableau
df.to_csv("macro_data_2016_2023.csv", index=False)

# 1. Sort by country and year
df_sorted = df.sort_values(["Country", "Year"])

# 2. Track previous income category
df_sorted["Prev_Income"] = df_sorted.groupby("Country")["Income_Category"].shift(1)

# 3. Mark if changed
df_sorted["Income_Changed"] = df_sorted["Income_Category"] != df_sorted["Prev_Income"]
df_sorted["Changed_To"] = df_sorted.apply(
    lambda row: row["Income_Category"] if row["Income_Changed"] else pd.NA, axis=1
)

# Optional: keep only the transitions
income_transitions = df_sorted[df_sorted["Income_Changed"] & df_sorted["Prev_Income"].notna()]
income_transitions.to_csv("income_category_transitions.csv", index=False)

# --- 8. Convert to long format ---
value_vars = [
    "GDP_per_capita", "Tax_revenue", "Inflation", "Debt",
    "GNI_per_capita", "Population", "Unemployment", "Interest"
]

df_long = df.melt(
    id_vars=["Country", "Year", "Income_Category"],
    value_vars=value_vars,
    var_name="Indicator",
    value_name="Value"
)

# --- 9. Save long-format file ---
df_long.to_csv("macro_data_long_2013_2023.csv", index=False)

In [8]:
import pandas as pd

# Load the multi-year merged data (assuming you've already created it)
df = pd.read_csv("macro_data_2016_2023.csv")

# 1. Pivot the data to wide format per indicator
pivoted = df.pivot_table(
    index=["Country", "Year"],
    values=["GDP_per_capita", "Tax_revenue", "Debt", "Inflation", "GNI_per_capita", 
            "Unemployment", "Interest", "Population"],
    aggfunc="first"
).reset_index()

# 2. Check for countries with complete data over all years and all indicators
# Group by Country and check if all data points (for all years) are present
complete_countries = (
    pivoted
    .groupby("Country")
    .apply(lambda group: group.dropna().shape[0] == len(group))
)
# Filter only countries with complete records
valid_countries = complete_countries[complete_countries].index.tolist()

# 3. Filter the original DataFrame
df_cleaned = df[df["Country"].isin(valid_countries)].reset_index(drop=True)

# 4. Save to new file (optional)
df_cleaned.to_csv("macro_data_cleaned_2013_2023.csv", index=False)


In [9]:
import pandas as pd

# Load your multi-year merged dataset
df = pd.read_csv("macro_data_2016_2023.csv")

# Define the full set of required years and indicators
required_years = set(str(y) for y in range(2013, 2024))
required_indicators = [
    "GDP_per_capita", "Tax_revenue", "Debt", "Inflation",
    "GNI_per_capita", "Unemployment", "Interest", "Population"
]

# Step 1: Pivot to wide format for easier checking
pivoted = df.pivot_table(
    index=["Country", "Year"],
    values=required_indicators,
    aggfunc="first"
).reset_index()

# Step 2: Drop any row that has missing data
pivoted_clean = pivoted.dropna()

# Step 3: Count valid years per country
country_year_counts = pivoted_clean.groupby("Country")["Year"].nunique()

# Step 4: Keep only countries that have all years
valid_countries = country_year_counts[country_year_counts == len(required_years)].index.tolist()

# Step 5: Filter the original dataframe
df_cleaned = df[df["Country"].isin(valid_countries)].reset_index(drop=True)

# Save cleaned version (optional)
df_cleaned.to_csv("macro_data_strictly_clean_2013_2023.csv", index=False)
